In [1]:
import tensorflow as tf
import os
from constants import UNKNOWN_FACES_PATH, MODEL_INPUT_IMAGE_DIMENSIONS, PATH_TO_FAILED_MODEL_EVALUATION_IMAGES, MODEL_0_FAILED_PATH, MODEL_1_FAILED_PATH, MODEL_2_FAILED_PATH, KNOWN_FACE_1_PATH, KNOWN_FACE_2_PATH, MODEL_PATH
from helpers import preprocess, deprocess_image, display_progressbar, model_prediction
import numpy as np
import time
from face_tracking import face_detection
from PIL import Image
import shutil

model = tf.keras.models.load_model(os.path.join(MODEL_PATH, 'model_best_with_josh_1.h5'))
required_bound=.8
certainty_bound=.1

#### Check model manually

In [2]:
face_detection(model=model, required_bound=required_bound, certainty_bound=certainty_bound)

#### Delete old files

In [2]:
for directory in os.listdir(PATH_TO_FAILED_MODEL_EVALUATION_IMAGES):
    if os.path.isdir(os.path.join(PATH_TO_FAILED_MODEL_EVALUATION_IMAGES, directory)):
        for file in os.listdir(os.path.join(PATH_TO_FAILED_MODEL_EVALUATION_IMAGES, directory)):
            file_path = os.path.join(PATH_TO_FAILED_MODEL_EVALUATION_IMAGES, directory, file)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print('Failed to delete %s. Reason: %s' % (file_path, e))

### Evaluate model on all data

#### Evaluate on photos of Stephen

In [ ]:
num_failures_0 = 0
t_orig = time.time()
total_photos = len(os.listdir(KNOWN_FACE_1_PATH))
total_processed = 0
t = time.time()
for face in os.listdir(KNOWN_FACE_1_PATH):
    face_path = os.path.join(KNOWN_FACE_1_PATH, face)
    im = deprocess_image(preprocess(face_path, MODEL_INPUT_IMAGE_DIMENSIONS))
    pred = model.predict(np.array([im]), verbose=0)[0]
    if  model_prediction(pred, [None, None], required_bound=required_bound, certainty_bound=certainty_bound) != "0":
        num_failures_0 += 1
        print("\rNumber of failures for stephen images: " + str(num_failures_0) + "; " + str(pred))
        im = Image.fromarray(im)
        im.save(os.path.join(MODEL_0_FAILED_PATH, face))
    total_processed += 1
    t = display_progressbar(t1=t, t_orig=t_orig, total_photos=total_photos, total_processed=total_processed)

#### Evaluate on photos of Josh

In [ ]:
num_failures_1 = 0
t_orig = time.time()
total_photos = len(os.listdir(KNOWN_FACE_2_PATH))
total_processed = 0
t = time.time()
for face in os.listdir(KNOWN_FACE_2_PATH):
    face_path = os.path.join(KNOWN_FACE_2_PATH, face)
    im = deprocess_image(preprocess(face_path, MODEL_INPUT_IMAGE_DIMENSIONS))
    pred = model.predict(np.array([im]), verbose=0)[0]
    if model_prediction(pred, [None, None], required_bound=required_bound, certainty_bound=certainty_bound) != "1":
        num_failures_1 += 1
        print("\rNumber of failures for josh images: " + str(num_failures_1) + "; " + str(pred))
        im = Image.fromarray(im)
        im.save(os.path.join(MODEL_1_FAILED_PATH, face))
    total_processed += 1
    t = display_progressbar(t1=t, t_orig=t_orig, total_photos=total_photos, total_processed=total_processed)

Failed
Number of failures for josh images: 1; [0.02210406 0.9999994  0.03867474]
Number of failures for josh images: 2; [0.10301451 0.99997616 0.05116026]
Number of failures for josh images: 3; [0.00519685 0.9999517  0.619703  ]
Number of failures for josh images: 4; [0.0175228  0.9999975  0.11863843]
Number of failures for josh images: 5; [0.07056522 0.99998724 0.05014376]
Number of failures for josh images: 6; [0.00466354 0.99999356 0.4148676 ]
Number of failures for josh images: 7; [2.4892428e-04 9.9878138e-01 9.9190760e-01]
Number of failures for josh images: 8; [0.00359049 0.99999905 0.26583707]
Number of failures for josh images: 9; [0.09204053 0.9999831  0.06100271]
Number of failures for josh images: 10; [0.0089413  0.99999833 0.17193225]
Number of failures for josh images: 11; [0.00645988 0.9999945  0.32991192]
Number of failures for josh images: 12; [2.5408930e-04 9.9999988e-01 5.8749443e-01]
Number of failures for josh images: 13; [0.00742598 0.99998486 0.4272935 ]
Number of

#### Evaluate on photos of unknown people

In [ ]:
num_failures_2 = 0
t_orig = time.time()
total_photos = len(os.listdir(UNKNOWN_FACES_PATH))
total_processed = 0
t = time.time()
for face in os.listdir(UNKNOWN_FACES_PATH):
    face_path = os.path.join(UNKNOWN_FACES_PATH, face)
    im = deprocess_image(preprocess(face_path, MODEL_INPUT_IMAGE_DIMENSIONS))
    pred = model.predict(np.array([im]), verbose=0)[0]
    if model_prediction(pred, [None, None], required_bound=required_bound, certainty_bound=certainty_bound) != "unknown":
        num_failures_2 += 1
        print("\rNumber of failures for unknown images: " + str(num_failures_2) + "; " + str(pred))
        im = Image.fromarray(im)
        im.save(os.path.join(MODEL_2_FAILED_PATH, face))
    total_processed += 1
    t = display_progressbar(t1=t, t_orig=t_orig, total_photos=total_photos, total_processed=total_processed)

In [ ]:
print("Total failures: " + str(num_failures_0 + num_failures_1 + num_failures_2))